The second dataset comes from Realtor [link:](https://www.realtor.com/) that has detailed real estate information that comes from MLS. [link:](http://www.mls.com/).

In [205]:
import re
import bs4 as bs
import urllib.request
import pandas as pd
import psycopg2
import time
import random

In [45]:
# sauce = urllib.request.urlopen('https://www.realtor.com/realestateandhomes-search/Manhattan_NY').read()

# soup = bs.BeautifulSoup(sauce, 'lxml')

# sauce

In [239]:
req = urllib.request.Request(
    #'https://www.realtor.com/realestateandhomes-search/Manhattan_NY',
    'https://www.realtor.com/realestateandhomes-search/Manhattan_NY/type-condo',
    data=None, 
    headers={
        #'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) App leWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53'
        #'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
        # 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
        
        #'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36',
        #'cookie':'gadsTest=test; split=n; split_tcv=175; __vst=3a22f8d4-832a-4926-b93b-352568953896; ab.storage.userId.7cc9d032-9d6d-44cf-a8f5-d276489af322=%7B%22g%22%3A%22visitor_3a22f8d4-832a-4926-b93b-352568953896%22%2C%22c%22%3A1603636015191%2C%22l%22%3A1603636015191%7D; ab.storage.deviceId.7cc9d032-9d6d-44cf-a8f5-d276489af322=%7B%22g%22%3A%2226090450-9a1d-d009-0499-8d717b57fbcb%22%2C%22c%22%3A1603636015201%2C%22l%22%3A1603636015201%7D; G_ENABLED_IDPS=google; s_ecid=MCMID%7C59012389654716554162096026837711734778; ajs_anonymous_id=%22dc5eeabe-2280-41a3-8a55-75ea87b143b5%22; __gads=ID=0929f623ce2f1a67:T=1603636016:S=ALNI_MauWE-k6boD0ZbKWTJIxVwz8Bjx9g; _fbp=fb.1.1603636016462.300088735; _ga=GA1.2.1909527488.1603636015; _ncg_id_=3fe0803a-e019-4a99-847a-a019bdc65b15; __qca=P0-580666646-1603636017277; _ncg_g_id_=1664a916-150d-475d-b4bd-dc00fbfdd426; threshold_value=43; clstr=; clstr_tcv=; vidoraUserId=pt70uohpj3bghlonvqmvooqo80ui5n; _parsely_visitor={%22id%22:%22pid=e7bc4720b1bbc695692eb97895ca1953%22%2C%22session_count%22:2%2C%22last_session_ts%22:1605300072982}; __ssn=7f2c8f15-0af0-43f8-a8fa-852ea62fcea7; __ssnstarttime=1607171130; __split=53; AMCVS_8853394255142B6A0A4C98A4%40AdobeOrg=1; _gid=GA1.2.557998350.1607171132; g_state={"i_p":1607775934424,"i_l":3}; user_activity=return; _ta=us~1~89df68bc35032d48c293adce8574aa42; _tac=false~self|not-available; leadid_token-27789EFE-7A9A-DB70-BB9B-97D9B7057DBB-01836014-7527-FD48-9B7F-1A40A9705CFE=93397DF6-C26C-A5DC-63B2-49873F01D286; srchID=d99b917efe234aee8ff7bfce4957f3f8; criteria=pg%3D1%26locSlug%3DManhattan_NY%26sprefix%3D%252Frealestateandhomes-search%26city%3DManhattan%26state_code%3DNY%26state_id%3DNY%26area_type%3Dcity%26search_type%3Dcity%26lat%3D40.7766419%26long%3D-73.9701871%26county_fips%3D36061%26county_fips_multi%3D36061%26loc%3DManhattan%252C%2520NY; QSI_SI_2a9qZZvRnb5FaZf_intercept=true; last_ran=1607202808415; last_ran_threshold=1607202808415; ab.storage.sessionId.7cc9d032-9d6d-44cf-a8f5-d276489af322=%7B%22g%22%3A%22351e0571-568d-0cec-38da-3ab624338e94%22%2C%22e%22%3A1607204608466%2C%22c%22%3A1607202808467%2C%22l%22%3A1607202808467%7D; QSI_CT={"avmGraphExperiment":2}; userStatus=return_user; automation=false; __edwssnstarttime=1607203327; bcc=false; bcvariation=SRPBCRR%3Av1%3Adesktop; last_viewed_property=%2Frealestateandhomes-detail%2F24-5th-Ave_New-York_NY_10011_B9181876428; _rdc-next_session=bXAwNVJqMDNvRlJtQjdYeGlnZXQxVHMrd2lDTHNKOGZsS0UzeEZoY1l6V0pOQVZ1ZENoMGNwODRKL0NYcEZpOEN0RTdlM1d4NEcyU2hoTXFWOU9pMlFCWGtqSW8zL2xtdkF2Z05nZENwOG1TZy9aNHlFNWZtQmhkaitkVVFlcDZrR0k2dTZYUFpHRlR0Zlg2ak96cE12U3JBMWltOUVuOVg3U21qVDJnR21hYlAxTmpyMjdvalhGSWJIQjhqdDNab3hsT0dqemx2SzkvTjhuYXB6NHE0bTFGTjVkdnNvN3VJWnNWelBmenhLbnpEUFhpaTJBYkZORllDSlFPQUVUQWlJbk5VM2QyWStGQldhc2lvSk5wVnc9PS0tMC9CMnljTXdLWS9FZXZ4dVhjdEtTdz09--88db6fb918a7801c117219da19ed15934e905ea3; AMCV_8853394255142B6A0A4C98A4%40AdobeOrg=-432600572%7CMCIDTS%7C18602%7CMCMID%7C59012389654716554162096026837711734778%7CMCAAMLH-1607819306%7C7%7CMCAAMB-1607819306%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1607221706s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.5.2; _uetsid=f791662036f411eb96141ff6ece87da5; _uetvid=f79189a036f411eb85e261625e04fd3a; QSI_HistorySession=https%3A%2F%2Fwww.realtor.com%2F~1607171131903%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F301-E-63rd-St-Apt-14J_New-York_NY_10065_M40260-30636~1607171224757%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083~1607185689796%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968~1607191898162%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083~1607191965195%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968~1607192094858%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083~1607202739968%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F1175-York-Ave-Apt-8C_New-York_NY_10065_M96570-92632~1607202766149%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F45-Tudor-City-Pl-Apt-220_New-York_NY_10017_M46492-96523~1607202823500%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F20-West-St-Apt-12M_New-York_NY_10004_M46154-65345~1607202828564%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F24-5th-Ave_New-York_NY_10011_B9181876428~1607203328290%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F24-5th-Ave-Apt-1023_New-York_NY_10011_M36799-86216~1607203369011%7Chttps%3A%2F%2Fwww.realtor.com%2Frealestateandhomes-detail%2F24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968~1607214507010; adcloud={%22_les_v%22:%22y%2Crealtor.com%2C1607216307%22}; _tas=8yg63euvslw; _ncg_sp_ses.cc72=*; _ncg_sp_id.cc72=3fe0803a-e019-4a99-847a-a019bdc65b15.1603636017.1.1607214507.1603636017.5ce5850f-80ff-46d4-a0ca-c50afdfcb9c6; _ppl_visit=uuid=06083035-ea79-4445-bd4b-008d8040dc71; _ppl_visitor=uuid=6ea87037-5230-4233-8fa8-fa2df166150c&birth=1605205660&count=9; reese84=3:DT0RYk7DORyeSD+cZqpMQQ==:Ikdl1eiJaRj80o7F1D/0s2vEaSe0B9hWiq5Y7DCK6bomOct0LhQ346iXdDWVG6WK7GEjLG1tbgJevk2nL+NDtxpRoGrdJy1as5i46WN44PAB1eH5k4DdVuBWqBTLzrfLoG8EtkhpU7sQdQCKof+ItjoNy+iHe/WRllqgl6Qb9o8Ysar8C17S4JWIwm4sdSomOTA8c8BGgSiqBSMywTAYc1cxRi2onRM6Di4jXi4li9PxOgnVM5MTVgZdJ0qMzZmtG4i3bkTg45R5+3TPMW4eluZF7ImrJ3HGf/TV2MB7Q5bpx9eaMFZfEC8TSnz2WcpCpaBMJjmdKN5HHmMPcv39BFWnQZ4hscAy9wNbGERYlaRUmiLdxqbMqlfr2VwD5V3SzFI+XVjEkH0z0ES5f28o8U9jI1TFnr5xuCM0Dq1sVGg=:mkCnB74FYDa1hpgZbZ4/T9+DFDE5yJ4peQRnuUt+Y58='
        
    }
)

f = urllib.request.urlopen(req)

sauce = f.read().decode('utf-8')

soup = bs.BeautifulSoup(sauce, 'lxml')

In [240]:
soup.title
# soup.find_all('p')

<title class="jsx-810260673">Condos for Sale in Manhattan, NY | realtor.com®</title>

In [229]:
for par in soup.find_all('p'):
    print(par.text)

Note: To increase accuracy, the keyword filter suggests the most commonly searched terms. Results may vary.
* To verify enrollment eligibility, contact the school or district directly.
* To verify enrollment eligibility, contact the school or district directly.
* To verify enrollment eligibility, contact the school or district directly.
* To verify enrollment eligibility, contact the school or district directly.
* To verify enrollment eligibility, contact the school or district directly.
A pre-approval letter from a lender makes your offer stronger.
So you are looking for multi family homes for sale in Manhattan, New York? Excellent choice! Realtor.com® wants to make sure you can search multi family homes with ease. That is why we have compiled a list of 99 multi family homes that are currently for sale within Manhattan residential boundaries, including open house listings. View each home individually and read the property details, which include the price, sales history, property tax, 

In [230]:
for url in soup.find_all('a'):
    print(url.get('href'))

/
/realestateforsale
/realestateandhomes-search/Manhattan_NY
/newhomecommunities/Manhattan_NY
/realestateandhomes-search/Manhattan_NY/shw-nc
/foreclosures/Manhattan_NY
/realestateandhomes-search/Manhattan_NY/show-open-house
https://www.seniorhousingnet.com/
/realestateandhomes-search/Manhattan_NY/overview
/soldhomeprices/Manhattan_NY
/propertyrecord-search/Manhattan_NY
/advice/buy/
/advice/sell/
/home-buying-checklist
/advice/buy/five-things-i-wish-i-had-known-as-a-first-time-homeowner/
/welcome/first-time-buyer
/advice/buy/
/realpeople/
/sell
/sell/sellers-marketplace
/sell
/tag/home-selling-guide/
/advice/sell/interviewing-listing-agents/
/advice/sell/home-maintenance-tasks-while-selling/
/advice/sell/
/rentals
/apartments/Manhattan_NY
https://www.seniorhousingnet.com/
/apartments/Manhattan_NY/affordable
/apartments/Manhattan_NY/luxury
/mortgage/tools/rent-or-buy-calculator/
//www.realtor.com/advice/rent/renting-myths/
/tag/pets/
/advice/rent/
/list-rental-property
/mortgage/
/mortga

In [231]:
soup.find_all('ul')

[<ul aria-label="Homes for Sale" role="menu"><li role="none"><a href="/realestateandhomes-search/Manhattan_NY" role="menuitem" title="">Manhattan Homes For Sale</a></li><li role="none"><a href="/newhomecommunities/Manhattan_NY" role="menuitem" title="">Manhattan New Homes Communities</a></li><li role="none"><a href="/realestateandhomes-search/Manhattan_NY/shw-nc" role="menuitem" title="">Manhattan New Construction and Plans</a></li><li role="none"><a href="/foreclosures/Manhattan_NY" role="menuitem" title="">Manhattan Foreclosures</a></li><li role="none"><a href="/realestateandhomes-search/Manhattan_NY/show-open-house" role="menuitem" title="">Manhattan Open Houses</a></li><li role="none"><a href="https://www.seniorhousingnet.com/" role="menuitem" title="">Senior Housing</a></li></ul>,
 <ul aria-label="Homes Values" role="menu"><li role="none"><a href="/realestateandhomes-search/Manhattan_NY/overview" role="menuitem" title="">Manhattan Housing Market</a></li><li role="none"><a href="/s

In [232]:
soup.findAll("ul", {"class": "jsx-2910884806 property-list list-unstyle"})

[<ul class="jsx-2910884806 property-list list-unstyle" data-testid="property-list-container"><li class="jsx-2423110403 component_property-card" data-testid="result-card"><div><div class="jsx-4044968377 pre-card-wrap"><div class="jsx-4044968377 broker-info"><div class="jsx-4044968377 ellipsis"><span class="jsx-4044968377">Brokered by<!-- --> </span><span class="jsx-4044968377" data-label="pc-brokered">Compass</span></div></div></div><div class="jsx-4195823209 card-box type-srp-result" data-id="9709824347" data-label="property-card" data-testid="property-card"><div class="jsx-4195823209 photo-wrap" data-testid="pc-photo-wrap" id="9709824347"><a data-testid="property-anchor" href="/realestateandhomes-detail/45-E-68th-St_New-York_NY_10065_M97098-24347" rel="noopener" target="_self"><picture><source srcset="https://ap.rdcpix.com/d05b2fca9dba45f5d8e5a094c13364f1l-m3269939171od-w480_h360.webp" type="image/webp"/><img alt="45 E 68th St, New York, NY 10065" class="fade top" data-atf="true" data

In [233]:
soup.findAll("div", {"class" : "jsx-4195823209 photo-wrap"})

[<div class="jsx-4195823209 photo-wrap" data-testid="pc-photo-wrap" id="9709824347"><a data-testid="property-anchor" href="/realestateandhomes-detail/45-E-68th-St_New-York_NY_10065_M97098-24347" rel="noopener" target="_self"><picture><source srcset="https://ap.rdcpix.com/d05b2fca9dba45f5d8e5a094c13364f1l-m3269939171od-w480_h360.webp" type="image/webp"/><img alt="45 E 68th St, New York, NY 10065" class="fade top" data-atf="true" data-fmp="true" data-label="pc-photo" data-src="https://ap.rdcpix.com/d05b2fca9dba45f5d8e5a094c13364f1l-m3269939171od-w480_h360.jpg" itemprop="image" src="https://ap.rdcpix.com/d05b2fca9dba45f5d8e5a094c13364f1l-m3269939171od-w480_h360.jpg"/></picture></a><div class="jsx-4195823209 save-wrap"><button aria-label="Save Listing" class="sc-fzoJus jooxVR save-btn" data-testid="save-button" type="button"><svg aria-label="heart" class="sc-fzqAui eoGDzK" data-testid="complex-svg-heart" height="40" tabindex="-1" viewbox="0 0 40 40" width="40" xmlns="http://www.w3.org/2000

In [234]:
divs = soup.findAll("div", {"class" : "jsx-4195823209 photo-wrap"})
for div in divs:
    children = div.findChildren()
    for child in children:
        print(child.get('href'))

/realestateandhomes-detail/45-E-68th-St_New-York_NY_10065_M97098-24347
None
None
None
None
None
None
None
/realestateandhomes-detail/216-Lafayette-St_New-York_NY_10012_M97405-39352
None
None
None
None
None
None
None
/realestateandhomes-detail/554-W-165th-St_New-York_NY_10032_M41641-09207
None
None
None
None
None
None
None
/realestateandhomes-detail/245-W-13th-St_New-York_NY_10011_M34330-89257
None
None
None
None
None
None
None
/realestateandhomes-detail/436-W-20th-St_New-York_NY_10011_M99968-51517
None
None
None
None
None
/realestateandhomes-detail/330-E-120th-St_New-York_NY_10035_M96936-37558
None
None
None
None
None
/realestateandhomes-detail/332-E-120th-St_New-York_NY_10035_M94989-67408
None
None
None
None
None
/realestateandhomes-detail/334-E-120th-St_New-York_NY_10035_M93359-07089
None
None
None
None
None
/realestateandhomes-detail/958-Avenue-of-the-Americas_New-York_NY_10001_M92569-43624
None
None
None
None
None
/realestateandhomes-detail/114-W-13th-St_New-York_NY_10011_M37761-77

In [11]:
links = soup.findAll("a", {"data-testid" : "property-anchor"})
for lnk in links:
    print(lnk.get('href'))

/realestateandhomes-detail/24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968
/realestateandhomes-detail/24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968
/realestateandhomes-detail/24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968
/realestateandhomes-detail/301-E-63rd-St-Apt-14J_New-York_NY_10065_M40260-30636
/realestateandhomes-detail/301-E-63rd-St-Apt-14J_New-York_NY_10065_M40260-30636
/realestateandhomes-detail/301-E-63rd-St-Apt-14J_New-York_NY_10065_M40260-30636
/realestateandhomes-detail/301-E-63rd-St-14HJ_New-York_NY_10065_M98514-60162
/realestateandhomes-detail/301-E-63rd-St-14HJ_New-York_NY_10065_M98514-60162
/realestateandhomes-detail/301-E-63rd-St-14HJ_New-York_NY_10065_M98514-60162
/realestateandhomes-detail/301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083
/realestateandhomes-detail/301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083
/realestateandhomes-detail/301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083
/realestateandhomes-detail/150-W-51st-St-Apt-831_New-York_

In [154]:
# single_url = 'https://www.realtor.com/' + '/realestateandhomes-detail/24-5th-Ave-Apt-710_New-York_NY_10011_M39925-68968'
# single_url = 'https://www.realtor.com/realestateandhomes-detail/301-E-63rd-St-Apt-14H_New-York_NY_10065_M35305-94083'
# single_url = 'https://www.realtor.com/realestateandhomes-detail/24-5th-Ave-Apt-1023_New-York_NY_10011_M36799-86216'
single_url = 'https://www.realtor.com//realestateandhomes-detail/252-South-St-Unit-9H_Manhattan_NY_10002_M38922-99399'
single_req = urllib.request.Request(
    single_url, 
    data=None, 
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
)

single_f = urllib.request.urlopen(single_req)

single_sauce = single_f.read().decode('utf-8')

single_soup = bs.BeautifulSoup(single_sauce, 'lxml')

In [155]:
# single_sauce

In [156]:
# single_soup.table

In [159]:
tbodys = single_soup.findAll("table")
tbodys

[<table class="table table-clear table-heading-unstyled table-school school-rating-lg">
 <thead>
 <tr>
 <th class="col-school-score text-align-center">Rating*</th>
 <th class="col-school-name">School Name</th>
 <th class="col-school-grades hidden-xxs hidden-xs">Grades</th>
 <th class="col-school-distance hidden-xxs hidden-xs">Distance</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td class="text-align-center">
 <span class="school-rating">10</span>
 </td>
 <td>
 <a data-omtag="ldp:tabs:schools:nearby:PublicElementary" href="/local/schools/Ps-184-Shuang-Wen-0751517831" target="_blank">Ps 184 Shuang Wen School</a>
 <span class="display-block fine-print"></span>
 </td>
 <td class="hidden-xxs hidden-xs">PK–8</td>
 <td class="hidden-xxs hidden-xs">0.3 mi</td>
 </tr>
 <tr>
 <td class="text-align-center">
 <span class="school-rating">6</span>
 </td>
 <td>
 <a data-omtag="ldp:tabs:schools:nearby:PublicMiddle" href="/local/schools/East-Side-Community-School-0751454721" target="_blank">East Side Communi

In [167]:
print(len(tbodys))
print(type(tbodys))
for tbody in tbodys:
    #print(tbody)
    #print(type(tbody))
    print('------------')
    s = str(tbody)

    if s.find("Price/Sq Ft") != -1:
        print(s)

4
<class 'bs4.element.ResultSet'>
------------
------------
<table class="table">
<thead>
<tr>
<th>Date</th>
<th>Event</th>
<th class="text-align-center">Price</th>
<th class="text-align-center hidden-xs hidden-xxs">Price/Sq Ft</th>
<th class="hidden-xs hidden-xxs">Source</th>
</tr>
</thead>
<tbody>
<tr>
<td>12/04/2020</td>
<td>Relisted</td>
<td class="text-align-center">$1,835,000</td>
<td class="text-align-center hidden-xs hidden-xxs">$1,587</td>
<td class="hidden-xs hidden-xxs">NestioNYC</td>
</tr>
<tr>
<td>12/04/2020</td>
<td>Listed</td>
<td class="text-align-center">$1,835,000</td>
<td class="text-align-center hidden-xs hidden-xxs">$1,587</td>
<td class="hidden-xs hidden-xxs">REBNY</td>
</tr>
<tr>
<td>09/30/2020</td>
<td>Listed</td>
<td class="text-align-center">$1,895,000</td>
<td class="text-align-center hidden-xs hidden-xxs">$1,639</td>
<td class="hidden-xs hidden-xxs">RealPlus</td>
</tr>
<tr>
<td>08/04/2020</td>
<td>Listed</td>
<td class="text-align-center">$1,895,000</td>
<td

In [28]:
print(type(tbodys[0]))

<class 'bs4.element.Tag'>


In [29]:
html_tables = pd.read_html(str(tbodys[0]))
html_tables

[         Date     Event     Price Price/Sq Ft          Source
 0  11/02/2020  Relisted  $835,000      $1,192        RealPlus
 1  10/19/2020    Listed  $835,000      $1,192        RealPlus
 2  09/29/2020    Listed  $835,000      $1,192             OLR
 3  08/21/2020    Listed  $835,000      $1,192           REBNY
 4  10/15/2013      Sold  $815,000      $1,207  DouglasElliman
 5  09/26/2012      Sold  $550,000           -   Public Record]

In [22]:
price_history_table_df = html_tables[0]

In [23]:
price_history_table_df.columns

Index(['Date', 'Event', 'Price', 'Price/Sq Ft', 'Source'], dtype='object')

In [33]:
price_per_sqft_spans = single_soup.findAll("span", {"class": "jsx-488154125 key"})
for s in price_per_sqft_spans:
    if s.text == 'Price per sqft':
        print(s.next_sibling.text)

$1,193


In [63]:
import urllib.parse
import urllib.request

# url = 'http://www.someserver.com/cgi-bin/register.cgi'
url = 'https://www.realtor.com/realestateandhomes-search/Manhattan_NY'
values = {'name' : 'Michael Foord',
          'location' : 'Northampton',
          'language' : 'Python' }

data = urllib.parse.urlencode(values)
data = data.encode('ascii') # data should be bytes
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as response:
   the_page = response.read()
the_page

HTTPError: HTTP Error 403: 

In [67]:
# 间隔10秒

print('11')

time.sleep(10)

print('12')

11
12


In [173]:
# connect postgresql
def get_postgresql_connect():
    conn = psycopg2.connect(host="127.0.0.1",user="postgres",password="123456",database="aim5001")
    return conn

def query_house_history(county='', state=''):
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from house_history_prices "
    if county and state:
        sql = sql + ' where county = %(q_county)s and state = %(q_state)s'
        cur.execute(sql, {'q_county':county, 'q_state':state})
    else:
        cur.execute(sql)
    
    rows = cur.fetchall()
    for row in rows:
        print(type(row))
        print(row)
        print(row[0])
        print(row[1])
        print(row[2])
        print(row[3])
        
    conn.close()

# query_house_history(county='New York', state='New York')
query_house_history()

<class 'tuple'>
(1, '123456789', 'New York', 'New York', 'Co-op', Decimal('1193.00'), 'Listed', datetime.date(2020, 11, 2))
1
123456789
New York
New York
<class 'tuple'>
(2, 'haha12345', 'New York', 'New York', 'House', Decimal('1192.00'), 'Relisted', datetime.date(2020, 11, 2))
2
haha12345
New York
New York
<class 'tuple'>
(3, 'haha12345', 'New York', 'New York', 'House', Decimal('1192.00'), 'Listed', datetime.date(2020, 10, 19))
3
haha12345
New York
New York
<class 'tuple'>
(4, 'haha12345', 'New York', 'New York', 'House', Decimal('907.00'), 'Sold', datetime.date(2013, 10, 15))
4
haha12345
New York
New York
<class 'tuple'>
(5, 'haha12345', 'New York', 'New York', 'House', Decimal('1192.00'), 'Relisted', datetime.date(2020, 11, 2))
5
haha12345
New York
New York
<class 'tuple'>
(6, 'haha12345', 'New York', 'New York', 'House', Decimal('1192.00'), 'Listed', datetime.date(2020, 10, 19))
6
haha12345
New York
New York
<class 'tuple'>
(7, 'haha12345', 'New York', 'New York', 'House', Decima

In [218]:
def query_links():
    """
        load links from Database
        
        return list of tuple
        
        For example:
        [('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-condo',
          'Houston',
          'Texas',
          'Condo'),
         ('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-multi-family-home',
          'Houston',
          'Texas',
          'Multi Family')]
    """
    
    conn= get_postgresql_connect()
    cur = conn.cursor()
    sql = " select * from query_links "
    
    cur.execute(sql)
    
    rows = cur.fetchall()
    
    conn.close()
    
    return rows

# test
query_links()

[('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-condo',
  'Houston',
  'Texas',
  'Condo'),
 ('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-multi-family-home',
  'Houston',
  'Texas',
  'Multi Family'),
 ('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-single-family-home',
  'Houston',
  'Texas',
  'Single Family'),
 ('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-townhome',
  'Houston',
  'Texas',
  'Single Townhome'),
 ('https://www.realtor.com/realestateandhomes-search/Los-Angeles_CA/type-condo',
  'Los Angeles',
  'California',
  'Condo'),
 ('https://www.realtor.com/realestateandhomes-search/Los-Angeles_CA/type-multi-family-home',
  'Los Angeles',
  'California',
  'Multi Family'),
 ('https://www.realtor.com/realestateandhomes-search/Los-Angeles_CA/type-single-family-home',
  'Los Angeles',
  'California',
  'Single Family'),
 ('hhttps://www.realtor.com/realestateandhomes-search/Los-Angeles_CA/type-tow

In [206]:
def convert_formatted_money_to_float(money):
    """
        money = '$1,192'
        
        return = 1192.0
    """
    print(money)
    #if money and money.find("-") == -1 and money.find("—") == -1:
    if money and money.find("$") != -1:
        s = money
        s = s.replace("$", "")
        s = s.replace(",", "")
    
        return float(s)

ff = convert_formatted_money_to_float('')
print(ff)

ff = convert_formatted_money_to_float('-')
print(ff)

ff = convert_formatted_money_to_float('$1,192')
print(ff)


None
-
None
$1,192
1192.0


In [197]:
def insert_history_prices(house_code, county, state, house_type, price_table_df):
    conn= get_postgresql_connect()
    cur = conn.cursor()
    
    sql = """
        INSERT INTO house_history_prices(house_code, county, "state", house_type, price_sqft, price_event, price_event_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
    
    for index, row in price_table_df.iterrows():

        p_sqft = convert_formatted_money_to_float(row['Price/Sq Ft'])
        
        if p_sqft is not None:
            values = (house_code, county, state, house_type, p_sqft, row['Event'], row['Date'])

            cur.execute(sql, values)
        
    conn.commit()
    print("Records created successfully")
    conn.close()

```python
         Date     Event     Price Price/Sq Ft          Source
 0  11/02/2020  Relisted  $835,000      $1,192        RealPlus
 1  10/19/2020    Listed  $835,000      $1,192        RealPlus
 2  09/29/2020    Listed  $835,000      $1,192             OLR
 3  08/21/2020    Listed  $835,000      $1,192           REBNY
 4  10/15/2013      Sold  $815,000      $1,207  DouglasElliman
 5  09/26/2012      Sold  $550,000           -   Public Record
```

In [198]:
test_dict = {"Date":['11/02/2020', '10/19/2020','10/15/2013'],
             "Event":['Relisted', 'Listed', 'Sold'],
             "Price":['$835,000', '$835,000', '$550,000'],
             "Price/Sq Ft":['$1,192', '-', '$907']}



In [199]:
test_df = pd.DataFrame(test_dict)
test_df

for index, row in test_df.iterrows():
    print(row['Event'], row['Price/Sq Ft'])

Relisted $1,192
Listed -
Sold $907


In [200]:
insert_history_prices('haha12345', 'New York', 'New York', 'House', test_df)

$1,192
-
$907
Records created successfully


In [179]:
random_second = random.randint(10, 20)

print('Sleep time: ', random_second)

Sleep time:  19


In [235]:
realtor_domain_name = 'https://www.realtor.com/'

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

def scrape_list_page(url_tuple):
    print('scrape url:', url_tuple)
    
    url = url_tuple[0]
    
    req = urllib.request.Request(
        url, 
        data=None, 
        headers={
            'User-Agent': user_agent
        }
    )

    f = urllib.request.urlopen(req)

    sauce = f.read().decode('utf-8')

    soup = bs.BeautifulSoup(sauce, 'lxml')

    links = soup.findAll("a", {"data-testid" : "property-anchor"})

    print('this page has links:', len(links))
    
    i = 0
    
    unique_hrefs = set()
    for lnk in links:
        unique_hrefs.add(lnk.get('href'))
    
    print('this page has unique links:', len(unique_hrefs))
    
    for href in unique_hrefs:
        
        random_second = random.randint(10, 20)
        
        sleep_time = 10 + random_second
        
        time.sleep(sleep_time)
        
        print('Sleep time: ', sleep_time)
        
        # add domain name
        single_soup = get_detial_page_html(realtor_domain_name + href)
        
        href_arr = href.split('/')

        if single_soup and len(href_arr) > 0:
            house_code = href_arr[-1]
        
            get_detail_page_history_prices_table(single_soup, house_code, url_tuple)
        i += 1
    print(i)

def get_detial_page_html(href):
    print('detial page link:' + href)
    single_req = urllib.request.Request(
    href, 
    data=None, 
    headers={
        'User-Agent': user_agent
    }
    )

    single_f = urllib.request.urlopen(single_req)

    single_sauce = single_f.read().decode('utf-8')

    single_soup = bs.BeautifulSoup(single_sauce, 'lxml')
    
    return single_soup


def get_detail_page_history_prices_table(single_soup, house_code, url_tuple):
    print('get history prices table')
    
    tbodys = single_soup.findAll("table")
    
    if len(tbodys) > 0:
        for tbody in tbodys:
            s = str(tbody)
            if s.find("Price/Sq Ft") != -1:
                html_tables = pd.read_html(s)
                if len(html_tables) > 0:
                    price_history_table_df = html_tables[0]
                    
                    print(price_history_table_df)
                    insert_history_prices(house_code, url_tuple[1], url_tuple[2], url_tuple[3], price_history_table_df)
            
    #tbodys = single_soup.findAll("table", {"class" : "jsx-2655468463"})
    #html_tables = pd.read_html(str(tbodys[0]))
    #price_history_table_df = html_tables[0]
    
    #print(price_history_table_df)

In [243]:
realtor_domain_name = 'https://www.realtor.com/'

def scrape_start():
    
    # step1: load query links from database
    urls = query_links()
    
    if len(urls) > 0:
        for url_tuple in urls:
            # step2:
            
            scrape_list_page(url_tuple)

scrape_start()

scrape url: ('https://www.realtor.com/realestateandhomes-search/Houston_TX/type-condo/pg-2', 'Houston', 'Texas', 'Condo')
this page has links: 126
this page has unique links: 42
Sleep time:  28
detial page link:https://www.realtor.com//realestateandhomes-detail/1555-Bering-Dr-128_Houston_TX_77057_M87680-14717
get history prices table
          Date          Event     Price Price/Sq Ft   Source
0   12/03/2020         Listed  $279,700        $168  Houston
1   11/13/2020         Listed  $282,000        $169  Houston
2   11/02/2020         Listed  $294,000        $176  Houston
3   10/05/2018  Price Changed  $310,000        $186  Houston
4   09/28/2018         Listed  $317,900        $191  Houston
5   05/26/2018         Listed  $316,900        $190  Houston
6   05/08/2018  Price Changed  $316,900        $190  Houston
7   04/03/2018         Listed  $324,000        $194  Houston
8   04/29/2016           Sold         -           —      NaN
9   09/24/2003           Sold         -           —   

Sleep time:  30
detial page link:https://www.realtor.com//realestateandhomes-detail/4723-Aftonshire-Dr-Apt-13_Houston_TX_77027_M82678-26223
get history prices table
         Date   Event     Price Price/Sq Ft   Source
0  12/02/2020  Listed  $174,500        $220  Houston
$220
Records created successfully
Sleep time:  21
detial page link:https://www.realtor.com//realestateandhomes-detail/2300-Old-Spanish-Trl-Apt-2105_Houston_TX_77054_M75418-15565
get history prices table
         Date          Event     Price Price/Sq Ft   Source
0  12/04/2020         Listed  $145,000        $160  Houston
1  11/27/2020  Price Changed  $145,000        $160  Houston
2  11/12/2020  Price Changed  $146,000        $161  Houston
3  09/30/2020         Listed  $149,900        $165  Houston
4  06/27/2013           Sold         -           —  Houston
5  04/26/2013         Listed  $110,000        $121  Houston
6  07/17/2009           Sold         -           —  Houston
7  05/06/2009         Listed  $104,900        

Sleep time:  26
detial page link:https://www.realtor.com//realestateandhomes-detail/5001-Woodway-Dr-Unit-805_Houston_TX_77056_M72501-49554
get history prices table
         Date          Event     Price Price/Sq Ft   Source
0  12/04/2020         Listed  $369,000        $241  Houston
1  04/07/2020  Price Changed  $369,000        $241  Houston
2  12/09/2019         Listed  $374,000        $244  Houston
3  09/05/2019         Listed  $385,000        $251  Houston
4  08/22/2013           Sold         -           —      NaN
5  09/13/2001           Sold         -           —      NaN
6  10/05/1999           Sold         -           —      NaN
$241
$241
$244
$251
—
—
—
Records created successfully
Sleep time:  23
detial page link:https://www.realtor.com//realestateandhomes-detail/260-El-Dorado-Blvd-Apt-1602_Webster_TX_77598_M83116-70507
get history prices table
         Date   Event    Price Price/Sq Ft   Source
0  12/04/2020  Listed  $78,000         $97  Houston
1  01/05/2018    Sold        -

this page has links: 126
this page has unique links: 42
Sleep time:  26
detial page link:https://www.realtor.com//realestateandhomes-detail/14230-Hillsboro-St_Houston_TX_77015_M84713-71517
get history prices table
         Date   Event     Price Price/Sq Ft   Source
0  07/04/2020  Listed  $385,000        $240  Houston
$240
Records created successfully
Sleep time:  26
detial page link:https://www.realtor.com//realestateandhomes-detail/907-W-Gray-St-Apt-4_Houston_TX_77019_M89622-32857
get history prices table
         Date          Event     Price Price/Sq Ft   Source
0  10/15/2020  Price Changed  $699,000        $231  Houston
1  08/03/2020         Listed  $750,000        $248  Houston
2  04/06/2006           Sold         -           —      NaN
3  09/25/1998           Sold         -           —      NaN
$231
$248
—
—
Records created successfully
Sleep time:  30
detial page link:https://www.realtor.com//realestateandhomes-detail/10317-Lake-Rd-14_Houston_TX_77070_M95714-56508
get history p

get history prices table
         Date          Event     Price Price/Sq Ft   Source
0  09/28/2020         Listed  $254,900        $130  Houston
1  07/06/2020         Listed  $254,900        $130  Houston
2  06/19/2020  Price Changed  $254,900        $130  Houston
3  03/11/2020         Listed  $264,900        $135  Houston
$130
$130
$130
$135
Records created successfully
Sleep time:  20
detial page link:https://www.realtor.com//realestateandhomes-detail/514-Woodland-St-Apt-5_Houston_TX_77009_M72604-09427
get history prices table
         Date          Event     Price Price/Sq Ft   Source
0  11/01/2020  Price Changed  $675,000        $246  Houston
1  07/15/2020  Price Changed  $699,000        $255  Houston
2  06/16/2020         Listed  $719,000        $262  Houston
$246
$255
$262
Records created successfully
Sleep time:  26
detial page link:https://www.realtor.com//realestateandhomes-detail/10317-Lake-Rd-11_Houston_TX_77070_M90788-08101
get history prices table
         Date          Ev

...

Sleep time:  22
detial page link:https://www.realtor.com//realestateandhomes-detail/1043-Maroney-Ln_Pacific-Palisades_CA_90272_M10206-80800
get history prices table
         Date   Event       Price Price/Sq Ft Source
0  12/07/2020  Listed  $3,350,000      $1,106   CLAW
1  06/15/2004    Sold  $1,870,000        $737    NaN
$1,106
$737
Records created successfully
Sleep time:  26
detial page link:https://www.realtor.com//realestateandhomes-detail/10354-Louisiana-Ave_W-Los-Angeles_CA_90025_M21872-02078
get history prices table
         Date   Event       Price Price/Sq Ft             Source
0  12/07/2020  Listed  $2,395,000      $1,083               CLAW
1  07/18/2018    Sold  $2,166,085        $980  SanDiegoCountySPN
2  07/16/2018  Listed  $2,175,000        $984  SanDiegoCountySPN
3  08/03/2012    Sold  $1,142,000        $661               CLAW
4  06/07/2012  Listed  $1,029,000        $595              ITECH
5  06/07/2012  Listed  $1,029,000        $595               CLAW
6  08/07/1998  

Sleep time:  20
detial page link:https://www.realtor.com//realestateandhomes-detail/675-Muskingum-Ave_Pacific-Palisades_CA_90272_M28410-58932
get history prices table
         Date   Event       Price Price/Sq Ft Source
0  12/07/2020  Listed  $2,995,000      $2,333   CLAW
1  06/28/1972    Sold     $57,500         $45    NaN
$2,333
$45
Records created successfully
42
scrape url: ('https://www.realtor.com/realestateandhomes-search/Los-Angeles_CA/type-townhome/pg-2', 'Los Angeles', 'California', 'Single Townhome')
this page has links: 126
this page has unique links: 42
Sleep time:  21
detial page link:https://www.realtor.com//realestateandhomes-detail/11733-Montana-Ave-Apt-215_Los-Angeles_CA_90049_M25857-12694
get history prices table
         Date   Event     Price Price/Sq Ft Source
0  12/06/2020  Listed  $995,000        $549  CRMLS
1  07/31/1985    Sold  $247,000        $136    NaN
$549
$136
Records created successfully
Sleep time:  29
detial page link:https://www.realtor.com//realesta

Sleep time:  22
detial page link:https://www.realtor.com//realestateandhomes-detail/901-E-42nd-St_Los-Angeles_CA_90011_M15997-95015
get history prices table
          Date     Event     Price Price/Sq Ft   Source
0   11/25/2020    Listed  $589,950        $293  CRISNET
1   11/21/2020    Listed  $615,000        $306     CLAW
2   03/15/2013    Listed  $295,000        $147     CLAW
3   09/30/2010  Relisted  $210,000        $104     CLAW
4   09/13/2010  Relisted  $210,000        $104     CLAW
5   08/04/2010  Relisted  $210,000        $104     CLAW
6   08/03/2010      Sold  $205,000        $102    ITECH
7   07/16/2010  Relisted  $210,000        $104     CLAW
8   07/01/2010  Relisted  $210,000        $104     CLAW
9   03/30/2010    Listed  $210,000        $104    ITECH
10  03/30/2010    Listed  $210,000        $104     CLAW
11  03/30/2010    Listed  $210,000        $104     CLAW
12  09/09/2002    Listed  $206,000        $102    ITECH
13  08/26/2002    Listed  $206,000           —    ITECH
$29

Sleep time:  24
detial page link:https://www.realtor.com//realestateandhomes-detail/13815-Victory-Blvd-Unit-2_Valley-Glen_CA_91401_M22370-47003
get history prices table
         Date     Event     Price Price/Sq Ft    Source
0  11/22/2020    Listed  $599,000        $402     CRMLS
1  12/02/2014      Sold  $355,000        $238       NaN
2  10/01/2014    Listed  $349,000        $234     CRMLS
3  10/01/2014    Listed  $349,000        $234      CLAW
4  10/12/2009      Sold  $300,000        $194  SoCalMLS
5  03/16/2009  Relisted  $379,000        $245  SoCalMLS
6  04/09/2008    Listed  $379,000        $245   CRISNET
7  04/09/2008    Listed  $379,000        $245  SoCalMLS
$402
$238
$234
$234
$194
$245
$245
$245
Records created successfully
Sleep time:  29
detial page link:https://www.realtor.com//realestateandhomes-detail/19226-Lanark-St_Reseda_CA_91335_M11046-22076
get history prices table
         Date   Event     Price Price/Sq Ft Source
0  11/24/2020  Listed  $939,900        $378  CSMAR
$3

$569
$483
$483
$472
$472
$320
$323
$323
$322
$323
$324
$325
$326
$335
$341
$348
$349
$348
$364
$376
$320
$391
$391
$410
$391
$333
$345
$345
$185
$142
Records created successfully
Sleep time:  30
detial page link:https://www.realtor.com//realestateandhomes-detail/533-E-36th-St_Los-Angeles_CA_90011_M26834-01250
get history prices table
         Date     Event     Price Price/Sq Ft Source
0  11/29/2020    Listed  $560,000        $296  CRMLS
1  04/07/2016    Listed  $405,000        $214  CRMLS
2  03/31/2015  Relisted  $399,000        $211  CRMLS
3  01/22/2015    Listed  $399,000        $211  CRMLS
4  02/26/1990      Sold   $36,000         $19    NaN
$296
$214
$211
$211
$19
Records created successfully
Sleep time:  28
detial page link:https://www.realtor.com//realestateandhomes-detail/6201-Valley-Circle-Blvd-Unit-15_Woodland-Hills_CA_91367_M15503-71387
get history prices table
         Date          Event     Price Price/Sq Ft     Source
0  12/04/2020         Listed  $649,999        $317   

In [211]:
s = 'https://www.realtor.com//realestateandhomes-detail/252-South-St-Unit-9H_Manhattan_NY_10002_M38922-99399'
s_arr = s.split('/')

if len(s_arr) > 0
    print(s_arr[-1])

252-South-St-Unit-9H_Manhattan_NY_10002_M38922-99399
